In [2]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, List
from shapely.geometry import Polygon
from ipywidgets import interact, FloatSlider, Label


# Exercise 1: **Side-Looking (Real Aperture) Radar**

*Explain the geometry associated with a Side-Looking Radar (SLR) and visualize the change of the pixel-size across the swath.*

A side-looking Radar observes in the direction perpendicular to the satellite track/direction of propagation. Its antenna is designed in such a way, that it scans a trapezoid-shaped area, whereas the trapezoid is rather narrow in the direction os satellite propagation (though widening the further away from the satelltie a target on the ground is). To achieve this, the antenna needs do be longer in the direction of satellite propagation than perpendicular to it. Thus, the beamwidth of such satellties solely depends on the antenna geometry itself. 

Usually side-looking real aperture satellites utilize pulses for range resolution. The pixel dimension in the direction of satellite porpagation is a function of the distance $R$:

$r_a\left(R\right) = 2R\tan\left(\dfrac{\beta_h}{2}\right) \approx R\beta_h$

Taking the angle $\beta_h$ (the azimuthal angle) to correspond to the beamwidth via $\beta_h = \dfrac{\lambda}{l}$ (where $\lambda$ is the wavelength and $l$ the antenna lenght in the direction of satellite propagation), one can further write:

$r_a\left(R\right)\approx R\dfrac{\lambda}{l}$

The pixel dimension orthogonal to the propagation direction of the satellite, the so-called range-width $r_{\rho}\left(\tau \right)$, is a function of the pulse duration $\tau$:

$r_{\rho}\left(\tau \right) = \dfrac{v\tau}{2\sin\left(\theta\right)}$

Here, $\theta$ denotes the polar angle, or incidence angle. The distance $R$ can be expressed as function of $\theta$ via trigonometric relations, as $R=H\tan\left(\theta\right)$, where $H$ is the satellites altitude.

Thus, the pixel can be constructed as follows:

                                                             (0|R + r_{rho})
                     (-r_a(R+r_{rho})/2|R+r_{rho}) x----------------x---------------x (+r_a(R+r_{rho})/2|R+r_{rho})
                                                    \               |              /
                                                     \              |             /
                                                      \             |  r_{rho}   /
                                                       \            |           /
                                                        \           |          / 
                                           (-r_a(R)/2|R) x----------x---------x (+r_a(R)/2|R)
                                                          \       (0|R)      /
                                                           \                /
                                                            \              /
                                                             \            /
                                                              \          /
                                                               \        /
                                                                \ beta /
                                                                 \    /
                                                                  \  /
                                                                   \/
                                                                  (0|0)

Whereas the four corners of the pixel are given by (CW, starting at bottom left corner):

$\left( \dfrac{-\lambda H \tan\theta}{2l}, H\tan\theta\right)$

$\left( \dfrac{-\lambda \left(H \tan\theta + \dfrac{v \tau}{2\sin\theta}\right)}{2l}, H \tan\theta + \dfrac{v \tau}{2\sin\theta}\right)$

$\left( \dfrac{+\lambda \left(H \tan\theta + \dfrac{v \tau}{2\sin\theta}\right)}{2l}, H \tan\theta + \dfrac{v \tau}{2\sin\theta}\right)$

$\left( \dfrac{+\lambda H \tan\theta}{2l}, H\tan\theta\right)$

The plot below depicts the pixel dimensions, assuming the satellite does not propagate and that the curvature of the incident wavefront can be approximated as a straight line (plane wave). 

## *How is the azimutal resolution of a SLR related to the height of the antenna above the ground?*

As stated above, the distance between antenna and ground $R$ (a.k.a. slant range) is a function of the polar angle $\theta$ and the satellite's altitude $H$. The azimuthal resolution is thus rewritable as:

$r_a\left(R\right)\approx R\dfrac{\lambda}{l} \approx \dfrac{H\lambda}{l}\tan\left(\theta\right)$

## *What are the pros/cons of using a side-looking geometry?*

* Pros:
  * easy detection of flat surfaces (no specular reflection component), as they appear as very dark
  * potentially scan of two swaths at once
  * signal interacts more with target (effective area of target larger, due to $\theta \neq 0$)

* Cons:
  * image distortions, that need to be correcetd for
  * the larger $\theta$, the less sensitive the antenna becomes/the lower its spatial resolution  $\left(\dfrac{\mathrm{Power}}{\mathrm{Area}} = \mathrm{const.}\right)$
  * spatial resolution limited by beamwidth, limited by antenna dimensions


## *What types of image distortions occur and how can we get rid of them?*

There are three types of image distortions:
* **slant-range distortion**:   
  * due to $\theta$, independent of target
  * geometric correction: ground range = slant range $R\left(\theta\right) \cdot \sin\theta$

* **foreshortening**:
  * due to target not being flat (& earth's curvature)  
  * the closer the slope of a part of the terrain gets to being perpendicular to the observation direction, the brighter this part in the image (**highlighting**)
  * correction: topography msut be known and according calculations done
  
* **shadowing/layover**:
  * due to target being very steep (& earth's curvature)     
  * either extreme case of foreshortening, in worst case shadow of e.g. mountain 'hides' information
  * correction: not possible





In [81]:
LAMBDA = 1e-3       #m
FACTOR = 1e3
SPEED_OF_LIGHT = 299792458 #m/s
PROPAGATION_VELOCITY = SPEED_OF_LIGHT / FACTOR
TAU = 5e-2             #s
ALTITUDE = 800e3    #m
ANTENNA_LENGTH = 1  #m
THETA_0 = 30

In [82]:
def get_pixel(theta, altitude, antenna_length, wavelength, tau, propagation_velocity):
    y1 = altitude * np.tan(theta)
    x1 = -wavelength * y1 / (2*antenna_length)

    y2 = altitude*np.tan(theta) + ((propagation_velocity*tau) / 2*np.sin(theta))
    x2 = -wavelength * y2 / (2*antenna_length)

    y3 = y2
    x3 = -1 * x2

    y4 = y1
    x4 = -1 * x1

    return Polygon([(x1, y1), (x2, y2), (x3, y3), (x4, y4)])


In [87]:


tau_slider = FloatSlider(value=TAU, min=0, max=10, step=0.1, description=r'tau [s]:')
theta_slider = FloatSlider(value=THETA_0, min=0, max=89, step=1, description='Theta [deg]:')


@interact(theta=theta_slider, tau=tau_slider)
def update(theta, tau):
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    # ax.set_aspect(1)
    x, y = get_pixel(np.deg2rad(theta), ALTITUDE, ANTENNA_LENGTH, LAMBDA, tau, PROPAGATION_VELOCITY).exterior.xy
    ax.plot(x, y, linewidth=2, color = 'black', label='pixel')
    ax.fill(x, y, color = 'grey')
    colors = ['blue', 'red', 'purple', 'gold']
    # for xx, yy, c in zip(x, y, colors):
    #     ax.scatter(xx, yy, color=c, s=50, label=f'({(xx/1000):.4f}km, {(yy/1000):.4f}km)')

    # Additional plot configurations
    ax.legend()
    ax.set_xlabel('Distance [m]', fontsize = 12)
    ax.set_ylabel('Distance [m]', fontsize = 12)
    # ax.set_xlim(-1010, 1010)
    fig.suptitle(f'H: {ALTITUDE/1000}km, lambda: {LAMBDA}m, v: c/{FACTOR}, l: {ANTENNA_LENGTH}m\ntheta: {theta}deg, tau: {tau}s', fontsize = 16)
    rar = x[3] - x[0]
    rat = x[2] - x[1]
    rr = y[1] - y[0]
    ax.set_title(r'$r_a \left(R\right) = $' + f'{rar:.4f}m, ' + r'$r_a \left(R+r_{\rho}\right) = $' + f'{rat:.4f}m, ' + r'$r_{\rho} = $' + f'{rr:.4f}m', fontsize = 16)
    plt.tight_layout()

interactive(children=(FloatSlider(value=30.0, description='Theta [deg]:', max=89.0, step=1.0), FloatSlider(val…